In [37]:
# import packages
import pandas as pd
import numpy as np
import pickle
from sklearn import metrics
from tensorflow import keras

In [3]:
# load test data
input_path = 'H:\RediMinds\VCQI'
#train = pd.read_csv(input_path+"\VCQI_clean_train.csv")
test = pd.read_csv(input_path+"\VCQI_clean_test.csv")

In [4]:
#x_train = train.drop(labels='INTRA_OP_COMPLICATIONS', axis = 'columns').copy()
#y_train = train['INTRA_OP_COMPLICATIONS'].copy()
x_test = test.drop(labels='INTRA_OP_COMPLICATIONS', axis = 'columns').copy()
y_test = test['INTRA_OP_COMPLICATIONS'].copy() 

In [5]:
# check % of pos labels in test set
#print('% pos labels train {:.2f}'.format(y_train.sum()/len(y_train)))
print('% pos labels test {:.2f}'.format(y_test.sum()/len(y_test)))

% pos labels test 0.05


### Load Models

In [39]:
model_path = 'output/models/'

In [19]:
# import Decision Tree Classifier
from joblib import load
DTC = load(model_path+'DTC.joblib')


In [20]:
# import Random Forest Classifer
from joblib import load
RFR = load(model_path+'RFR.joblib')

In [21]:
# import Random Forest Classifer
from joblib import load
Logit = load(model_path+'Logit.joblib')

In [40]:
# load Neural Network Model
nn_model = keras.models.load_model(model_path+'nn_model.h5')

W0904 17:02:03.174817  9480 deprecation.py:323] From H:\Anaconda3\lib\site-packages\tensorflow\python\keras\metrics.py:1792: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Generate Predictions for the Test set

In [22]:
# Generate prediction for the Decsiion Tree Model
results_DTC = pd.DataFrame(DTC.predict(x_test), columns=['pred_label'])
results_DTC['pred_prob'] =  pd.DataFrame(DTC.predict_proba(x_test))[1]
results_DTC['true_label'] = np.array(y_test)

In [23]:
# Generate prediction for the Random Forest Model
results_RFR = pd.DataFrame(RFR.predict(x_test), columns=['pred_label'])
results_RFR['pred_prob'] =  pd.DataFrame(RFR.predict_proba(x_test))[1]
results_RFR['true_label'] = np.array(y_test)

In [32]:
# Generate prediction for the Random Forest Model

# load list of categorical variables
with open (input_path+'\cat_col', 'rb') as fp:
    cat_col = pickle.load(fp)

# Create dummy variables
logit_test = x_test.copy()
for col in cat_col:
    if not col == 'INTRA_OP_COMPLICATIONS':
        logit_test = logit_test.merge(pd.get_dummies(logit_test[col], prefix=col), how = 'left', left_index = True, right_index =True) # create dummy variables
        logit_test = logit_test.drop(labels = col, axis = 'columns') # drop original variables



# Generate prediction for the Logitic Regressions Model
results_Logit = pd.DataFrame(Logit.predict(logit_test), columns=['pred_label'])
results_Logit['pred_prob'] =  pd.DataFrame(Logit.predict_proba(logit_test))[1]
results_Logit['true_label'] = np.array(y_test)

In [50]:
# Generate prediction for Neural Network Model

# import Random Forest Classifer
from joblib import load
stdc = load(model_path+'nn_stdc.joblib')

#To prepare test data we will use test with one-hotcoded variables
nn_test = logit_test.copy()
float_col = test.select_dtypes('float').columns
nn_test[float_col] = pd.DataFrame(stdc.transform(nn_test[float_col]),columns=float_col).copy()

# Generate prediction for the Neural Network Model
results_nn = pd.DataFrame(nn_model.predict(nn_test), columns=['pred_prob'])
results_nn['pred_label'] =  results_nn['pred_prob'].apply(lambda x: 1 if x>=0.5 else 0)
results_nn['true_label'] = np.array(y_test)